In [39]:
##########################################
import numpy as np                       # NumPy pour le traitement de tableaux et matrices
import cv2                               # OpenCV pour la vision par ordinateur
from ultralytics import YOLO             # Import de la classe YOLO du module Ultralytics
import os                                # Module pour interagir avec le système d'exploitation
import time                              # Module pour mesurer le temps d'exécution
from PIL import Image   
import matplotlib.pyplot  as plt 
import easyocr
from datetime import datetime
reader = easyocr.Reader(['fr','en']) # this needs to run only once to load the model into memory                    

In [40]:
#  Chargement des modèles YOLO
##############################################
model_immat = YOLO("yolov8n.pt")

In [41]:
# results = model_immat(["plate-1/test/images/plate-153-_jpg.rf.434f34d5b883f43807ae1b4880788130.jpg"])
# results

In [42]:
# for r in results:
#     im_array = r.plot()  # plot a BGR numpy array of predictions
#     im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
#     # im.show()  # show image
#     x1, y1, x2, y2 = r.boxes.xyxy[0]
#     x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convert to int values
#     im= np.array(im)
    
#     print(im.shape) # Print image shape
#     cv2.imshow("original", im)
    
#     # Cropping an image
#     cropped_image = im[y1:y2, x1:x2]
    
#     # Display cropped image
#     cv2.imshow("cropped", cropped_image)
    
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

# Vidéo

In [43]:
import pandas as pd
import sqlite3
from functools import wraps
bdd_path ="./db.sqlite3"
con = sqlite3.connect(bdd_path)
con.execute("""CREATE TABLE IF NOT EXISTS Immat
			(
	Immatriculation TEXT ,
	Date TEXT NOT NULL
	)""")
con.commit()
con.close()


def connection(func):
    """
    Décorateur qui établit une connexion à une base de données SQLite,
    exécute une fonction et commit les modifications
    avant de fermer la connexion.

    Args:
        func (function): La fonction à décorer.

    Returns:
        function: La fonction décorée.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        con = sqlite3.connect(bdd_path)
        func(con, *args, **kwargs)
        con.commit()
        con.close()

    return wrapper
@connection
def import_data(con: sqlite3.Connection,data:dict):
    """
    Importe les données d'un fichier CSV dans une table de la base de données,
    en évitant les doublons.

    Args:
        con (sqlite3.Connection): La connexion à la base de données.
    """
    table = "Immat"
    df = pd.DataFrame.from_dict(data)
    existing_rows = pd.read_sql_query(
        f"""SELECT {','.join(df.columns)} FROM {table} WHERE Immatriculation = "{data['Immatriculation'][0]}" """, con)
    if existing_rows.empty:
        df.to_sql(table, con=con, if_exists="append", index=False)
    else:
        

In [44]:

video_path = "./videos/mycarplate.mp4"


from PIL import Image



# Initialize video capture
cap = cv2.VideoCapture(video_path)
liste_pred =[]
output_texts = "test"
h_to_add,w_to_add = 800,100
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    # If there's no frame to capture, break out of the loop
    if not ret:
        break
    frame = cv2.resize(frame,(800,600))
    frame_copy = frame.copy()
    try:
        cv2.putText(frame_copy,output_texts,(20, 20),fontFace=1,fontScale=1,color=(189,0,75),thickness=2)
    except:
        print("###############################")
        print(output_texts)
        print("###############################")
        output_texts = "Erreur"
    finally:
        cv2.putText(frame_copy,output_texts,(20, 20),fontFace=1,fontScale=1,color=(189,0,75),thickness=2)
    # Perform detection on the frame
    results = model_immat(frame, stream= True, line_width=1, show_conf=False, save=True)
    zone = np.array([[0, 400], [0+h_to_add, 400],[0+h_to_add, 400+w_to_add], [0, 400+w_to_add]])
    zone = zone.reshape((-1, 1, 2))
    cv2.polylines(frame_copy,[zone],color=(0,0,0),isClosed=True)
    for r in results:
        boxes = r.boxes
        names = r.names                                   #classes names list
        confs = r.boxes.conf.float().cpu().tolist()  
        for box , conf in zip(boxes, confs):
            if conf > 0.65:
                label = str(names[0] + " "+ str(conf))

                x1, y1, x2, y2 = box.xyxy[0].int().tolist()  # convert to int values and extract box coordinates
                immat_centre = (int((x1 + x2)/2),int((y1+y2)/2))
                cropped_image = frame[y1:y2, x1:x2]  # Crop the frame using the box coordinates
                # Display original frame
                result = cv2.pointPolygonTest(zone, immat_centre, False)
                if result == 1 :
                    try :
                        output_texts = reader.readtext(cropped_image)
                        output_texts = str(output_texts[0][1])
                        


                    
                        cv2.putText(frame_copy,label,(x1, y1),fontFace=1,fontScale=1,color=(189,0,75),thickness=2)
                        cv2.circle(frame_copy,immat_centre,2,color=(0,0,255))
                    except:
                        pass
                
                # Display cropped image
                cv2.imshow("Cropped", cropped_image)
            
    import_data(data={
        "Immatriculation": [output_texts],
        "Date": [datetime.now().strftime("%d-%m-%y %H-%M-%S")]
    })
    # Check if 'q' is pressed to exit
    cv2.imshow("Original", frame_copy)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()
cv2.destroyAllWindows()



0: 608x800 (no detections), 76.0ms
Speed: 4.0ms preprocess, 76.0ms inference, 1.5ms postprocess per image at shape (1, 3, 608, 800)
Results saved to runs/detect/predict21

0: 608x800 (no detections), 69.0ms
Speed: 2.3ms preprocess, 69.0ms inference, 1.3ms postprocess per image at shape (1, 3, 608, 800)
Results saved to runs/detect/predict21

0: 608x800 (no detections), 69.4ms
Speed: 3.0ms preprocess, 69.4ms inference, 1.2ms postprocess per image at shape (1, 3, 608, 800)
Results saved to runs/detect/predict21

0: 608x800 1 plate, 69.3ms
Speed: 3.1ms preprocess, 69.3ms inference, 2.6ms postprocess per image at shape (1, 3, 608, 800)
Results saved to runs/detect/predict21

0: 608x800 1 plate, 68.9ms
Speed: 3.1ms preprocess, 68.9ms inference, 1.6ms postprocess per image at shape (1, 3, 608, 800)
Results saved to runs/detect/predict21

0: 608x800 1 plate, 69.1ms
Speed: 2.9ms preprocess, 69.1ms inference, 1.8ms postprocess per image at shape (1, 3, 608, 800)
Results saved to runs/detect/pr